# Week 8: Text Generation Models
## How ChatGPT-Style AI Works

**Today's Goals:**
1. Understand how text generation models work
2. Use small GPT models that run on free Colab
3. Control generation with parameters (temperature, length, etc.)
4. Explore what these models can and can't do

---

## Part 1: How Text Generation Works

Text generation models predict **one word at a time**.

```
Input: "The cat sat on the"
Model thinks: What word comes next?
  - "mat" (30% likely)
  - "floor" (25% likely)
  - "couch" (20% likely)
  - etc.
Output: "mat"

Then it continues:
Input: "The cat sat on the mat"
Output: "and"
...
```

This is called **autoregressive generation**.

## Setup

In [ ]:
# Install required libraries
!pip install transformers -q
!pip install torch -q

print("Libraries installed!")

---
## Part 2: Your First Text Generator

We'll use **DistilGPT-2** - a smaller, faster version of GPT-2 that works great on free Colab.

In [ ]:
from transformers import pipeline

# Load a text generation model
# distilgpt2 is 82M parameters - small and fast!
generator = pipeline("text-generation", model="distilgpt2")

print("Text generator loaded! (DistilGPT-2 - 82M parameters)")

In [ ]:
# Generate some text!
prompt = "Once upon a time"

result = generator(prompt, max_length=50)

print(f"Prompt: {prompt}")
print(f"\nGenerated:")
print(result[0]['generated_text'])

### Try Different Prompts!

In [ ]:
# Try these prompts
prompts = [
    "The robot walked into the room and",
    "In the year 2050, humans will",
    "The best advice I ever received was",
    "Scientists discovered that"
]

for prompt in prompts:
    print("=" * 50)
    print(f"Prompt: {prompt}")
    result = generator(prompt, max_length=60, num_return_sequences=1)
    print(f"\nGenerated: {result[0]['generated_text']}")
    print()

---
## Part 3: Controlling Generation

You can control how the model generates text using parameters:

### Temperature: Creativity Control

- **Low temperature (0.1-0.5)**: More predictable, "safer" outputs
- **High temperature (0.8-1.5)**: More creative, random outputs

In [ ]:
prompt = "The meaning of life is"

print("Comparing temperatures:\n")

# Low temperature - predictable
print("--- Temperature = 0.3 (predictable) ---")
result = generator(prompt, max_length=40, temperature=0.3, do_sample=True)
print(result[0]['generated_text'])
print()

# High temperature - creative
print("--- Temperature = 1.2 (creative) ---")
result = generator(prompt, max_length=40, temperature=1.2, do_sample=True)
print(result[0]['generated_text'])

### Max Length: How Much Text to Generate

In [ ]:
prompt = "The future of AI is"

print("Comparing lengths:\n")

for length in [20, 50, 100]:
    print(f"--- max_length = {length} ---")
    result = generator(prompt, max_length=length)
    print(result[0]['generated_text'])
    print()

### Multiple Outputs: Get Different Versions

In [ ]:
prompt = "My favorite hobby is"

# Generate 3 different completions
results = generator(
    prompt, 
    max_length=40, 
    num_return_sequences=3,
    do_sample=True,
    temperature=0.9
)

print(f"Prompt: {prompt}\n")
for i, result in enumerate(results, 1):
    print(f"Version {i}: {result['generated_text']}")
    print()

---
## Part 4: The Bigger Model - GPT-2

Let's try the full GPT-2 small model (124M parameters):

In [ ]:
# Load the original GPT-2 (small)
gpt2_generator = pipeline("text-generation", model="gpt2")

print("GPT-2 loaded! (124M parameters)")

In [ ]:
# Compare DistilGPT-2 vs GPT-2
prompt = "Artificial intelligence will change the world by"

print("Comparing models on the same prompt:\n")
print(f"Prompt: {prompt}\n")

print("--- DistilGPT-2 (82M params) ---")
result1 = generator(prompt, max_length=60)
print(result1[0]['generated_text'])
print()

print("--- GPT-2 (124M params) ---")
result2 = gpt2_generator(prompt, max_length=60)
print(result2[0]['generated_text'])

---
## Part 5: Important Limitations!

These small models have limitations you should know:

### What They CAN Do:
- Complete sentences
- Generate creative text
- Follow patterns in the prompt

### What They CAN'T Do Well:
- Answer questions accurately
- Follow complex instructions
- Remember context from far back
- Reason about facts

**ChatGPT is MUCH bigger** (175 billion+ parameters vs our 82-124 million!)

In [ ]:
# See the limitations - asking a factual question
prompt = "The capital of France is"

result = generator(prompt, max_length=30)
print("Trying a factual prompt:")
print(result[0]['generated_text'])

# Sometimes it gets it right, sometimes not!
# This is why bigger models like ChatGPT are more reliable

---
## Part 6: Creative Writing Fun!

These models are great for creative writing experiments:

In [ ]:
def write_story(opening, length=150, temperature=0.8):
    """Generate a story from an opening line."""
    result = generator(
        opening,
        max_length=length,
        temperature=temperature,
        do_sample=True,
        num_return_sequences=1
    )
    return result[0]['generated_text']

# Generate a story!
story = write_story("It was a dark and stormy night when the mysterious stranger arrived at")
print("Generated Story:")
print("=" * 50)
print(story)

In [ ]:
# Try different genres
genres = {
    "Sci-Fi": "The spaceship landed on the alien planet, and the astronauts saw",
    "Mystery": "Detective Smith examined the clue carefully. It was",
    "Romance": "She looked into his eyes and realized that",
    "Horror": "The old house creaked as something moved in the darkness. It was"
}

for genre, opening in genres.items():
    print(f"\n{'='*50}")
    print(f"Genre: {genre}")
    print(f"{'='*50}")
    story = write_story(opening, length=80)
    print(story)

---
## Part 7: Understanding Why AI Says Weird Things

Sometimes AI generates strange or nonsensical text. Why?

1. **It's just predicting the next word** - no real understanding
2. **Training data patterns** - it learned from internet text
3. **Randomness** - high temperature means more random choices
4. **No fact-checking** - it doesn't know what's true

In [ ]:
# Watch the model generate some questionable content
# (because it's just pattern matching, not reasoning)

prompt = "The moon is made of"

print("Let's see what the model thinks the moon is made of:\n")
for i in range(3):
    result = generator(prompt, max_length=30, do_sample=True, temperature=1.0)
    print(f"{i+1}. {result[0]['generated_text']}")

---
## Part 8: Build a Text Generator Tool

In [ ]:
def creative_writer(prompt, style="normal"):
    """
    Generate text with different styles.
    
    Styles:
    - 'conservative': Safe, predictable
    - 'normal': Balanced
    - 'creative': Wild and random
    """
    settings = {
        "conservative": {"temperature": 0.3, "max_length": 60},
        "normal": {"temperature": 0.7, "max_length": 80},
        "creative": {"temperature": 1.2, "max_length": 100}
    }
    
    s = settings[style]
    
    result = generator(
        prompt,
        max_length=s["max_length"],
        temperature=s["temperature"],
        do_sample=True,
        num_return_sequences=1
    )
    
    print(f"Style: {style.upper()}")
    print(f"Temperature: {s['temperature']}")
    print("-" * 40)
    print(result[0]['generated_text'])
    return result[0]['generated_text']

# Try it!
test_prompt = "The secret to happiness is"

for style in ["conservative", "normal", "creative"]:
    print("\n" + "=" * 50)
    creative_writer(test_prompt, style)

---
## Parameter Reference

| Parameter | What it does | Typical values |
|-----------|-------------|----------------|
| `max_length` | Max tokens to generate | 50-200 |
| `temperature` | Creativity (higher = more random) | 0.3-1.2 |
| `do_sample` | Enable randomness | True/False |
| `num_return_sequences` | How many outputs | 1-5 |
| `top_k` | Consider only top K words | 10-50 |
| `top_p` | Consider words in top P probability | 0.9-0.95 |

---
## Checklist: What You Learned Today

- [ ] How text generation works (next word prediction)
- [ ] Using DistilGPT-2 and GPT-2 models
- [ ] Controlling output with temperature and other parameters
- [ ] Why AI sometimes says weird or wrong things
- [ ] The difference between small and large language models

---

## Looking Ahead: Next Week

Next week we'll explore **datasets**:
- Finding datasets on Hugging Face
- Loading and exploring data
- Understanding how datasets are used for training

**Homework (optional):**
- Generate stories with different prompts
- Experiment with temperature settings
- Save your favorite generations to GitHub!

---

*Youth Horizons AI Researcher Program - Level 2*